In [2]:
import plotly.figure_factory as ff

import numpy as np
import pandas as pd
import os
from datetime import date
from datetime import timedelta

url = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_confirmed_usafacts.csv'
popUrl = 'https://usafactsstatic.blob.core.windows.net/public/data/covid-19/covid_county_population_usafacts.csv'

confirmed_pd = pd.read_csv(url, index_col=False)
population_pd = pd.read_csv(popUrl, index_col=False)

sunday = date.today()
day = sunday.weekday()

while(day != 6):
    sunday = sunday - timedelta(days=1)
    day = sunday.weekday()

TE = sunday
TE = TE.strftime("%#m/%#d/%y")
TS = sunday - timedelta(days=7)
TS = TS.strftime("%#m/%#d/%y")
LE = sunday - timedelta(days=7)
LE = LE.strftime("%#m/%#d/%y")
LS = sunday - timedelta(days=14)
LS = LS.strftime("%#m/%#d/%y")

thisIR = []
lastIR = []
strFIPS = []
highIR = []
risk = []

population = population_pd['population']
lastStart = confirmed_pd[LS]
lastEnd = confirmed_pd[LE]
thisStart = confirmed_pd[TS]
thisEnd = confirmed_pd[TE]
FIPS = confirmed_pd['countyFIPS']
county = confirmed_pd['County Name'] + ', ' + confirmed_pd['State']

combined_pd = pd.DataFrame(list(zip(FIPS, county, population, lastStart, lastEnd, thisStart, thisEnd)), 
                           columns = ['FIPS', 'County', 'Population', 'Last Start', 'Last End', 'This Start','This End'])
combined_pd = combined_pd[combined_pd['Population'] != 0].reset_index(drop=True)

combined_pd['FIPS'] = combined_pd['FIPS'].astype('int64', copy=True)

for x in range(0,len(combined_pd)):
    thisIR.append((combined_pd['This End'][x] - combined_pd['This Start'][x])/combined_pd['Population'][x]*100000)
    lastIR.append((combined_pd['Last End'][x] - combined_pd['Last Start'][x])/combined_pd['Population'][x]*100000)
    if thisIR[x] > lastIR[x]:
        highIR.append(thisIR[x])
    else: 
        highIR.append(lastIR[x])
    strFIPS.append(str(combined_pd['FIPS'][x]).rjust(5,'0'))

combined_pd['FIPS'] = strFIPS
combined_pd['Last IR'] = lastIR
combined_pd['This IR'] = thisIR
combined_pd['Incidence Rate'] = highIR

mean = combined_pd['This IR'].mean()
std = combined_pd['This IR'].std()

for x in range(0,len(combined_pd)):
    if combined_pd['Incidence Rate'][x]<mean + std:
        risk.append('LOW')
    elif combined_pd['Incidence Rate'][x]>=mean + std and combined_pd['Incidence Rate'][x]<=mean+2*std:
        risk.append('MEDIUM')
    elif combined_pd['Incidence Rate'][x]>mean+2*std:
        risk.append('HIGH')
    else: 
        risk.append('ERROR')

combined_pd['Risk'] = risk

#display(missing_pd[missing_pd['_merge']!='both'])

#colorscale = ["#CCFFCC","#99FF99","#66FF66","#00FF00","#99FF00","#CCFF00","#FFFF00","#FFCC00","#FF6600","#FF0000"]
colorscale = ["#F1D2D6","#F4C2CB","#F2AAB7","#E98598","#D4596F","#BC394B","#9F292C", "#7A2F3D"]

endpts = list(np.linspace(1, mean+2*std, len(colorscale) - 1))
fips = combined_pd['FIPS'].tolist()
values = combined_pd['Incidence Rate'].tolist()

fig = ff.create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colorscale,
    show_state_data=True, 
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9,
    plot_bgcolor='#000033', paper_bgcolor='#000033',state_outline={'color': "#333333", 'width':0.5}
)

fig2 = ff.create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colorscale,scope=['Alaska'],
    show_state_data=True,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9,
    plot_bgcolor='#000033', paper_bgcolor='#000033'
)

fig3 = ff.create_choropleth(
    fips=fips, values=values,
    binning_endpoints=endpts,
    colorscale=colorscale,scope=['Hawaii'],
    show_state_data=False,
    show_hover=True, centroid_marker={'opacity': 0},
    asp=2.9,
    plot_bgcolor='#000033', paper_bgcolor='#000033'
)


fig.layout.template = None
fig2.layout.template = None
fig3.layout.template = None
fig.update_layout(margin=dict(l=120, r=0, b=95, t=60), showlegend=False)
fig.update_layout(font_color="white",legend=dict(yanchor='bottom', y = .05, xanchor='right', x = .95, font=dict(color="white")))
fig2.update_layout(margin=dict(l=20, r=0, b=0, t=0),showlegend=False)
fig3.update_layout(margin=dict(l=0, r=0, b=20, t=0),showlegend=False)

os.chdir(r'C:\Users\aj1ro\Documents\Incidence Map')
print('working..')
fig.write_html('first_figure.html',auto_open=True)
fig2.write_html('second_figure.html',auto_open=True)
fig3.write_html('third_figure.html',auto_open=True)
print('done')

ImportError: geopandas, pyshp and shapely must be installed for this figure factory.

Run the following commands to install the correct versions of the following modules:

```
$ pip install geopandas==0.3.0
$ pip install pyshp==1.2.10
$ pip install shapely==1.6.3
```
If you are using Windows, follow this post to properly install geopandas and dependencies:http://geoffboeing.com/2014/09/using-geopandas-windows/

If you are using Anaconda, do not use PIP to install the packages above. Instead use conda to install them:

```
$ conda install plotly
$ conda install geopandas
```